<a href="https://colab.research.google.com/github/fanyang2012/Grid-Studio/blob/master/Master_thesis_Chapter_5_Time_series_forecasting_Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data preprocessing for time series forecasting

## 1.1 Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1.2 Import dataset from clustering results

In [ ]:
def clustering(data, n): 

    num = cluster_num[cluster_num['cluster'] == n]['Skugroupid']
    clustering = clustering_result[clustering_result['cluster'] == n].reset_index()
    clustering = clustering.drop(columns = ['index'])
    for i in range(int(num)): 
        clusters = clustering['Skugroupid'][i]
        data.append(clusters)
    print('length of clustering: {} \n'.format(len(data)))
    
    return data

In [ ]:
def cluster(data,name): 

    for i in range(len(name)): 
        clusterid = new_groupdata[new_groupdata['Skugroupid'] == name[i]]
        data = data.append(clusterid)
    
    return data

In [ ]:
new_groupdata = pd.read_csv('new_groupdata')
clustering_result = pd.read_csv('clustering_result_4') # 4 clusters from fast moving items
cluster5 = pd.read_csv('slow_data') # 1 cluster which belongs to slow moving items
cluster_num = clustering_result.groupby('cluster')['Skugroupid'].count().reset_index()

In [ ]:
for i in range(1,5):
        #all skugroupids in the cluster
        globals()["clustering" + str(i)] = []
        globals()["clustering" + str(i)] = clustering(globals()["clustering" + str(i)], i-1)
        #data in the cluster
        globals()["cluster" + str(i)] = pd.DataFrame(columns=['Skugroupid', 'Day', 'Pick volume'])
        globals()["cluster" + str(i)] = cluster(globals()["cluster" + str(i)], globals()["clustering" + str(i)])

## 1.3 Select time series from cluster1, cluster3 and cluster5

In [ ]:
ts1 = cluster1[cluster1['Skugroupid'] == 111010]
ts1['Day'] = pd.to_datetime(ts1['Day'])
ts1.index = np.arange(ts1.shape[0])

In [ ]:
sku = cluster3[cluster3['Skugroupid'] == 513030]
sku['Day'] = pd.to_datetime(sku['Day'])
sku.index = np.arange(sku.shape[0])

In [ ]:
slow1 = cluster5[cluster5['Skugroupid'] == 117050]
slow1['Day'] = pd.to_datetime(slow1['Day'])
slow1.index = np.arange(slow1.shape[0])

## 1.4 Plot each time series

In [ ]:
def line_chart(data): 

    plt.figure(figsize=(20,6))
    plt.plot(data['Day'], data['Pick volume'], color='blue', marker='.', linestyle='-', linewidth=0.5)
    xmin,xmax = plt.xlim()
    ymin,ymax = plt.ylim()
    plt.grid()
    plt.title('Overview of data')
    plt.show()

In [ ]:
for data in [slow1, ts1, sku]:
    line_chart(data)

## 1.5 Basic statistical characteristics

In [ ]:
table = pd.DataFrame({'Time Series':['ts1', 'sku', 'slow1']})
table['mean'] = [data['Pick volume'].mean() for data in [ts1, sku, slow1]]
table['median'] = [data['Pick volume'].median() for data in [ts1, sku, slow1]]
table['std'] = [data['Pick volume'].std() for data in [ts1, sku, slow1]]
table['max'] = [data['Pick volume'].max() for data in [ts1, sku, slow1]]
table['min'] = [data['Pick volume'].min() for data in [ts1, sku, slow1]]
table['sum'] = [data['Pick volume'].sum() for data in [ts1, sku, slow1]]

## 1.6 Overview of changes based on each year, month and day

In [ ]:
def date_features(data): 
    
    data['Day'] = pd.to_datetime(data['Day'])
    
    data['Date_Day'] = data['Day'].dt.day
    data['Date_weekday'] = data['Day'].dt.weekday
    data['Date_week'] = data['Day'].dt.isocalendar().week.astype('int64')
    data['Date_month'] = data['Day'].dt.month
    data['Date_year'] = data['Day'].dt.year  
    
    return data

In [ ]:
for data in [slow1, ts1, sku]:
    data = date_features(data)

In [ ]:
def plot_change(data, time): 

    plt.figure(figsize=(20,8))
    plt.plot(data[time], data['ts1'], label = 'ts1', color = 'b')
    plt.plot(data[time], data['sku'], label = 'sku', color = 'r')
    plt.plot(data[time], data['slow1'], label = 'slow1', color = 'g')
    plt.legend()
    plt.show()

### 1.6.1 Overview of changes based on each year

In [ ]:
trend = pd.DataFrame({'year': [2015, 2016, 2017, 2018, 2019]})
trend['ts1'] = ts1.groupby('Date_year')['Pick volume'].mean().tolist()
trend['sku'] = sku.groupby('Date_year')['Pick volume'].mean().tolist()
trend['slow1'] = slow1.groupby('Date_year')['Pick volume'].mean().tolist()

In [ ]:
# add mean of year in 'table'
table['year_mean'] = [trend[col].mean() for col in ['ts1', 'sku', 'slow1']]
table['year_std'] = [trend[col].std() for col in ['ts1', 'sku', 'slow1']]

In [ ]:
plot_change(trend, 'year')

### 1.6.2 Overview of changes based on each month

In [ ]:
month = pd.DataFrame({'month': np.arange(1,13)})
month['ts1'] = ts1.groupby('Date_month')['Pick volume'].mean().tolist()
month['sku'] = sku.groupby('Date_month')['Pick volume'].mean().tolist()
month['slow1'] = slow1.groupby('Date_month')['Pick volume'].mean().tolist()

In [ ]:
# add mean of month in 'table'
table['month_mean'] = [month[col].mean() for col in ['ts1', 'sku', 'slow1']]
table['month_std'] = [month[col].std() for col in ['ts1', 'sku', 'slow1']]

In [ ]:
plot_change(month, 'month')

### 1.6.3 Overview of changes based on each day

In [ ]:
day = pd.DataFrame({'day': np.arange(1,32)})
day['ts1'] = ts1.groupby('Date_Day')['Pick volume'].mean().tolist()
day['sku'] = sku.groupby('Date_Day')['Pick volume'].mean().tolist()
day['slow1'] = slow1.groupby('Date_Day')['Pick volume'].mean().tolist()

In [ ]:
# add mean of day in 'table'
table['day_mean'] = [day[col].mean() for col in ['ts1', 'sku', 'slow1']]
table['day_std'] = [day[col].std() for col in ['ts1', 'sku', 'slow1']]

# 2. Feature engineering

In [ ]:
class CallableBaseTransformerMixin(BaseEstimator, TransformerMixin):
  """
  This basic Mixin allows you to create a stateless transformer from a Callable objects by inheriting from it.
  Stateless in this case means that the fit()-step of the transformer is merely a passthrough and
  the transformation is independent of the fit()-step.
  For instance removing a predetermined string from every column in a pd.DataFrame would be an operation like this,
  because it does not inherently require any knowledge from the data that must be acquired in the fit()-step.

  """

  def fit(self, X: Iterable, y: Optional[Iterable] = None) -> "CallableBaseTransformerMixin":
  """
  The fit function of a Callable transformer is always a passthrough because it is stateless.

  Args:
  X: data
  y: labels

  Returns:
  returns a pointer to itself

  """
    return self

  def transform(self, X: Iterable) -> Iterable:

  """
  The transform() step of a Transformer inheriting from this MixIn will be a simple mapping of the call step.
  Essentially __call__() and transform() will do the same thing.

  Args:
  X: The data to be transformed. This object is simply passed through to the __call__()-function, which must
  have a compatible interface of course.


  Returns:
  The output of the __call__() function.

  """
    return self(X)

In [ ]:
import datetime
import itertools
from copy import deepcopy
from typing import Any, Dict, List, Optional, Tuple, Union

import attr
import holidays
import numpy as np
import pandas as pd
from attr import attrib, attrs

GERMAN_STATE_ABBRIVATIONS_MAPPING = {
    "Baden-Württemberg": "BW",
    "Bayern": "BY",
    "Berlin": "BE",
    "Brandenburg": "BB",
    "Bremen": "HB",
    "Hamburg": "HH",
    "Hessen": "HE",
    "Mecklenburg-Vorpommern": "MV",
    "Niedersachsen": "NI",
    "Nordrhein-Westfalen": "NW",
    "Rheinland-Pfalz": "RP",
    "Saarland": "SL",
    "Sachsen": "SN",
    "Sachsen-Anhalt": "ST",
    "Schleswig-Holstein": "SH",
    "Thüringen": "TH",
}


@attrs(auto_attribs=True, frozen=True, slots=True)
class RemoveWeekday(CallableBaseTransformerMixin):
    """
    Removes all rows with a given weekday
    Args:
        weekday: the weekday (0=Monday, 6=Sunday)
        date_column: the name of the date column
    """

    _weekday: int = 5
    _date_column: str = "date"

    def __call__(self, data: pd.DataFrame):
        return data[data[self._date_column].dt.weekday != self._weekday]


@attrs(auto_attribs=True, frozen=True, slots=True)
class GroupedColumnShifter(CallableBaseTransformerMixin):
    """
    This transformer shifts given columns stepwise backwards by within a given range, while the rest of the data is
    duplicated for each step. This is performed while the data is grouped by a given list of columns.

    Args:
        shift_columns:              the set of columns to be shifted
        group_columns:              the set of columns to group the data by while performing the shift
        shift_identifier_column:    the name of the column that identifies the step size of the shift
        shift_range:                the range of steps to be performed

    """

    _shift_columns: Tuple[Tuple[str, int], ...]
    _shift_identifier_column: str
    _shift_range: Tuple[int, int]
    _group_columns: Optional[Tuple[str, ...]] = None

    def __call__(self, data: pd.DataFrame):
        df_list = [data]
        for i in np.arange(self._shift_range[0], self._shift_range[1] + 1, 1):
            df = deepcopy(data)
            if self._group_columns:
                df = df.groupby(list(self._group_columns)).apply(self._shift, i)
            else:
                df = self._shift(df, i)

            df[self._shift_identifier_column] = i
            df_list.append(df)
        result = pd.concat(df_list)
        return result

    def _shift(self, data: pd.DataFrame, shift_value: int):
        for column, shift_step in self._shift_columns:
            if column in data.columns:
                i = int(shift_value / shift_step) * shift_step
                data[column] = data[column].shift(i)
        return data


def state_subset(instance, attribute, value):
    return set(value).issubset(GERMAN_STATE_ABBRIVATIONS_MAPPING.values())


@attrs(auto_attribs=True, frozen=True, slots=True)
class UTCTimestampToDate(CallableBaseTransformerMixin):
    """
    Transforms UTCTimestamp to Pandas datetime
    Args:
        timezone: the timezone of the date
        unit: the unit of teh timestamp
        column: the name of the timestamp column
        name: the name of the new date column

    """

    _timezone: str
    _unit: str
    _column: str
    _name: str

    def __call__(self, data: pd.DataFrame):
        data[self._name] = pd.to_datetime(data[self._column], unit=self._unit)
        data[self._name] = data[self._name].dt.tz_localize("UTC").dt.tz_convert(self._timezone)
        data[self._name] = pd.to_datetime(data[self._name].dt.date)
        return data


@attrs(auto_attribs=True, frozen=True, slots=True)
class Substract(CallableBaseTransformerMixin):
    """
    Calculates the difference of th enumeic columns
    Args:
        column_a: the first column
        column_b: the second column
        name: the name of the resulting column

    """

    _column_a: str
    _column_b: str
    _name: str = "diff"

    def __call__(self, data: pd.DataFrame):
        data[self._name] = data[self._column_a] - data[self._column_b]
        return data


@attrs(auto_attribs=True, frozen=True, slots=True)
class PublicHolidayDeriver(CallableBaseTransformerMixin):
    """
    This deriver returns a boolean feature: True is the given date is a public holiday in any of the given states,
    False if not.
    Args:
        states:         List of states to check for public holidays, in None is given, it will look in all states
        date_column:    the name of the date column, needs to be given in datetime
        holiday_column: the name of the new feature column
        how:            if 'any' (default) it is true if the date is a holiday in any of the states, if 'all' it is
                        True if the date is  a holiday in all of the states
    """

    _date_column: str = "date"
    _holiday_column: str = "public_holiday"
    _states: Tuple[str, ...] = attrib(default=tuple(GERMAN_STATE_ABBRIVATIONS_MAPPING.values()), validator=state_subset)
    _how: str = attrib(default="any", validator=attr.validators.in_(["any", "all"]))

    def __call__(self, data: pd.DataFrame) -> pd.DataFrame:
        start = data[self._date_column].min()
        end = data[self._date_column].max()
        holidays_de = []
        if self._how == "any":
            for state in self._states:
                holidays_state = holidays.Germany(years=list(np.arange(start.year, end.year + 1, 1)), prov=state)
                holidays_de.append(holidays_state)
            all_holidays = set(itertools.chain.from_iterable(holidays_de))
        else:
            all_holidays = holidays.Germany(years=list(np.arange(start.year, end.year + 1, 1)), prov=self._states)

        data[self._holiday_column] = data[self._date_column].isin(all_holidays)
        return data


In [ ]:
holiday_pipeline_feature = Pipeline(
    [("timestamp to date",
                UTCTimestampToDate(column='Day', name='Day', unit="ms", timezone="Europe/Berlin"),
            ),
        (
            "school holiday deriver",
            SchoolHolidayDeriver(
                states=None, holiday_column=f"school_holiday", date_column='Day', how="any"
            ),
        ),
        (
            f"count till school holiday",
            CountTillEvent(date_column='Day', event_column=f"school_holiday", name=f"countTillSchoolHoliday"),
        ),
        (
            f"count since school holiday",
            CountSinceEvent(
                date_column='Day', event_column=f"school_holiday", name=f"countSinceSchoolHoliday"
            )),
        (
            f"public holiday deriver",
            PublicHolidayDeriver(
                date_column='Day',
              #  states=None,
                holiday_column=f"public_holiday",
                how="all")),
        (
            f"count till public holiday",
            CountTillEvent(
                date_column='Day',
                event_column=f"public_holiday",
                name=f"countTillPublicHoliday",
            )),
        (
            f"count since public holiday",
            CountSinceEvent(
                date_column='Day',
                event_column=f"public_holiday",
                name=f"countSincePublicHoliday",
            ))  
    ])

In [ ]:
dif_pipeline_feature = Pipeline(
        [
            (
                "difference month - quarter",
                Substract("MeanPickCountLast28Days", "MeanPickCountLast84Days", "Diff_meanPickCountLast28_84Days"),
            ),
            (
                "difference week - month",
                Substract("MeanPickCountLast7Days", "MeanPickCountLast28Days", "Diff_meanPickCountLast7_28Days"),
            ),
            (
                "difference weekday - 4weekdays",
                Substract("PickCountLastWeekday", "MeanPickCount4Weekdays", "Diff_meanPickCountLast1_4weekdays"),
            )
        ]
    )

In [ ]:
def feature_extraction(data, n=1): 
    
    data['Day'] = pd.to_datetime(data['Day'])

    #Add features with date features
    data['Date_Day'] = data['Day'].dt.day
    data['Date_weekday'] = data['Day'].dt.weekday
    data['Date_week'] = data['Day'].dt.isocalendar().week.astype('int64')
    data['Date_month'] = data['Day'].dt.month
    data['Date_year'] = data['Day'].dt.year
    data['Date_is_year_start'] = data['Day'].dt.is_year_start
    data['Date_is_year_end'] = data['Day'].dt.is_year_end
    data['Date_is_quarter_start'] = data['Day'].dt.is_quarter_start
    data['Date_is_quarter_end'] = data['Day'].dt.is_year_end
    data['Date_is_month_start'] = data['Day'].dt.is_month_start
    data['Date_is_month_end'] = data['Day'].dt.is_month_end    
    
    #Add holiday features
    data['school_holiday'] = np.nan
    data['countTillSchoolHoliday'] = np.nan
    data['countSinceSchoolHoliday'] = np.nan
    data['public_holiday'] = np.nan
    data['countTillPublicHoliday'] = np.nan
    data['countSincePublicHoliday'] = np.nan

    data = holiday_pipeline_feature.fit_transform(data)

    #Transfer holiday value and others with boolean of true/false into integer 0/1
    data['public_holiday'] = data['public_holiday'].apply(lambda x: int(x == True))
    data['school_holiday'] = data['school_holiday'].apply(lambda x: int(x == True))
    data['Date_is_year_start'] = data['Date_is_year_start'].apply(lambda x: int(x == True))
    data['Date_is_year_end'] = data['Date_is_year_end'].apply(lambda x: int(x == True))
    data['Date_is_quarter_start'] = data['Date_is_quarter_start'].apply(lambda x: int(x == True))
    data['Date_is_quarter_end'] = data['Date_is_quarter_end'].apply(lambda x: int(x == True))
    data['Date_is_month_start'] = data['Date_is_month_start'].apply(lambda x: int(x == True))
    data['Date_is_month_end'] = data['Date_is_month_end'].apply(lambda x: int(x == True))  


    #Add lag value features with mean / std of pick count in last days
    data['MeanPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).mean().shift(n).bfill())
    data['StdPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).std().shift(n).bfill())
    data['MedianPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).median().shift(n).bfill())
    data['MaxPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).max().shift(n).bfill())
    data['MinPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).min().shift(n).bfill())
    data['SumPickCountLast7Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=7,min_periods=1).sum().shift(n).bfill()) 
    
    data['MeanPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).mean().shift(n).bfill())
    data['StdPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).std().shift(n).bfill())
    data['MedianPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).median().shift(n).bfill())
    data['MaxPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).max().shift(n).bfill())
    data['MinPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).min().shift(n).bfill())
    data['SumPickCountLast28Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=28,min_periods=1).sum().shift(n).bfill())     
    
    data['MeanPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).mean().shift(n).bfill())
    data['StdPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).std().shift(n).bfill())
    data['MedianPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).median().shift(n).bfill())
    data['MaxPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).max().shift(n).bfill())
    data['MinPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).min().shift(n).bfill())
    data['SumPickCountLast84Days'] = data.groupby('Skugroupid')['Pick volume'].apply(lambda x: x.rolling(window=84,min_periods=1).sum().shift(n).bfill()) 

    #Add lag values features with mean / std of pick count in last weekdays
    data['PickCountLastWeekday'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=1, min_periods=1).mean().shift(int(n/7)+1).bfill())
    
    data['MeanPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).mean().shift(int(n/7)+1).bfill())
    data['StdPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).std().shift(int(n/7)+1).bfill())
    data['MedianPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).median().shift(int(n/7)+1).bfill())    
    data['MaxPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).max().shift(int(n/7)+1).bfill())
    data['MinPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).min().shift(int(n/7)+1).bfill()) 
    data['SumPickCount4Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=4, min_periods=1).sum().shift(int(n/7)+1).bfill())    
    
    data['MeanPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).mean().shift(int(n/7)+1).bfill())
    data['StdPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).std().shift(int(n/7)+1).bfill())
    data['MedianPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).median().shift(int(n/7)+1).bfill())    
    data['MaxPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).max().shift(int(n/7)+1).bfill())
    data['MinPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).min().shift(int(n/7)+1).bfill())   
    data['SumPickCount6Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=6, min_periods=1).sum().shift(int(n/7)+1).bfill())
    
    data['MeanPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).mean().shift(int(n/7)+1).bfill())
    data['StdPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).std().shift(int(n/7)+1).bfill())
    data['MedianPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).median().shift(int(n/7)+1).bfill())    
    data['MaxPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).max().shift(int(n/7)+1).bfill())
    data['MinPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).min().shift(int(n/7)+1).bfill())    
    data['SumPickCount8Weekdays'] = data.groupby(['Skugroupid', 'Date_weekday'])['Pick volume'].apply(lambda x: x.rolling(window=8, min_periods=1).sum().shift(int(n/7)+1).bfill())
    
    #Add difference features
    data["Diff_meanPickCountLast28_84Days"] = np.nan
    data["Diff_meanPickCountLast7_28Days"] = np.nan
    data["Diff_meanPickCountLast1_4weekdays"] = np.nan

    data = dif_pipeline_feature.fit_transform(data)

    #Sort values 
    data = data.sort_values(['Skugroupid','Day'])

    #Convert the string in the column of Skugroupid into integer
    data['Skugroupid'] = data['Skugroupid'].astype(int)
    
    return data

# 3. Performance metrics

In [ ]:
def frequency_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
"""
This metric computes the rate of missed events. An event is defined as a value >= 0.5 a non-event is defined
as a value < 0.5. The values of y_pred and y_true are rounded internally, but there will be no side effects, since
the function operates on deepcopies exclusively.

Args:
y_true: array-like of shape = (n_samples) or (n_samples, 1)
Ground truth (correct) target values.
y_pred : array-like of shape = (n_samples) or (n_samples, 1)
Estimated target values.
Returns:
rate of incorrect identifications of events (y_true >= 1) and non-events (y_true = 0), value is bounded between
0 and 1, with 1 meaning all events are identified wrongly, 0.0 the all events are detected correctly and 0.5 meaning a
(more or less) random result.

"""
# deecopy labels and predictions for rounding
  y_true = np.array(y_true)[:]
  y_pred = np.array(y_pred)[:]

  y_true[y_true >= 0.5] = 1
  y_true[y_true < 0.5] = 0
  y_pred[y_pred >= 0.5] = 1
  y_pred[y_pred < 0.5] = 0

  false_pred = sum(y_true != y_pred)
  result = false_pred / len(y_true)
  return result

In [ ]:
from typing import Callable, Dict, Optional, Union

import numpy as np
from numpy import corrcoef
from sklearn.metrics import mean_absolute_error, r2_score

def mean_absolute_percentage_error(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    sample_weight: Optional[np.ndarray] = None,
    multioutput: Optional[str] = "uniform_average",
) -> Union[float, np.ndarray]:
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape, weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == "raw_values":
            return output_errors
        elif multioutput == "uniform_average":
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)


def median_absolute_percentage_error(y_true: np.ndarray, y_pred: np.ndarray, epsilon: Optional[float] = None) -> float:
    if epsilon is None:
        epsilon = np.finfo(np.float64).eps
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    mdape = np.median(ape)
    return mdape


def median_absolute_percentage_error_shifted(
    y_true: np.ndarray, y_pred: np.ndarray, add_value: Optional[float] = None
) -> float:
    if add_value is None:
        add_value = 1
    y_true = y_true + add_value
    y_pred = y_pred + add_value
    return median_absolute_percentage_error(y_true, y_pred)


def mean_absolute_percentage_error_shifted(
    y_true: np.ndarray, y_pred: np.ndarray, add_value: Optional[float] = None
) -> float:
    if add_value is None:
        add_value = 1
    y_true = y_true + add_value
    y_pred = y_pred + add_value
    return float(mean_absolute_percentage_error(y_true, y_pred))

In [ ]:
import sklearn.metrics as metrics

mean_absolute_percentage_error_shifted(result['y_true'], result['y_pred']) # MAPE
median_absolute_percentage_error_shifted(result['y_true'], result['y_pred']) # MdAPE
metrics.r2_score(result['y_true'], result['y_pred']) # R squared
metrics.mean_absolute_error(result['y_true'], result['y_pred']) # MAE
metrics.mean_squared_error(result['y_true'], result['y_pred']) # MSE